In [4]:
from __future__ import annotations

from typing import Any, Dict, List, Optional
import requests

BASE_URL = "https://api.elections.kalshi.com/trade-api/v2"

def fetch_markets(
    status: str = "open",
    limit: int = 100,
    cursor: Optional[str] = None,
    series_ticker: Optional[str] = None,
    mve_filter: Optional[str] = "exclude", 
    timeout_s: int = 10,
) -> Dict[str, Any]:
    """
    Fetch markets from Kalshi (public market data).

    Returns the raw JSON response (dict). The markets list is typically in response["markets"].
    """
    url = f"{BASE_URL}/markets"
    params: Dict[str, Any] = {"limit": limit}

    if status:
        params["status"] = status
    if mve_filter:
        params["mve_filter"] = mve_filter
    if cursor:
        params["cursor"] = cursor
    if series_ticker:
        params["series_ticker"] = series_ticker

    resp = requests.get(url, params=params, timeout=timeout_s)
    resp.raise_for_status()
    return resp.json()


def search_markets_progressive(
    query: str,
    mve_filter: str = "exclude",
    page_limit: int = 200,
    initial_pages: int = 5,
    max_pages: int = 100,
    target_results: int = 100,
    status: str = "open",
) -> list[dict]:

    q = query.strip().lower()
    if not q:
        return []
    
    results = []
    cursor = None
    pages_checked = 0

    # phase 1: scan first few pages
    while pages_checked < initial_pages:
        raw = fetch_markets(limit=page_limit, cursor=cursor,
                             status=status, mve_filter=mve_filter)
        markets = raw.get("markets", [])
        for m in markets:
            if q in (m.get("title","").lower() + m.get("ticker","").lower()):
                results.append(m)
                if len(results) >= target_results:
                    return results
        cursor = raw.get("cursor") or raw.get("next_cursor")
        pages_checked += 1
        if not cursor:
            return results

    # phase 2: continue if needed
    while pages_checked < max_pages and len(results) < target_results:
        raw = fetch_markets(limit=page_limit, cursor=cursor,
                             status=status, mve_filter=mve_filter)
        markets = raw.get("markets", [])
        for m in markets:
            if q in (m.get("title","").lower() + m.get("ticker","").lower()):
                results.append(m)
                if len(results) >= target_results:
                    return results
        cursor = raw.get("cursor") or raw.get("next_cursor")
        pages_checked += 1
        if not cursor:
            break

    return results

In [11]:
results = search_markets_progressive("trump")

In [12]:
print(results[30])

{'can_close_early': True, 'close_time': '2029-01-21T15:00:00Z', 'created_time': '2026-02-04T01:12:31.847367Z', 'custom_strike': {'Person': 'Kristi Noem'}, 'early_close_condition': 'This market will close and expire early if the event occurs for any person.', 'event_ticker': 'KXCABOUT-29JAN', 'expected_expiration_time': '2029-01-21T15:00:00Z', 'expiration_time': '2029-01-21T15:00:00Z', 'expiration_value': '', 'last_price': 24, 'last_price_dollars': '0.2400', 'latest_expiration_time': '2029-01-21T15:00:00Z', 'liquidity': 872376, 'liquidity_dollars': '8723.7600', 'market_type': 'binary', 'no_ask': 81, 'no_ask_dollars': '0.8100', 'no_bid': 76, 'no_bid_dollars': '0.7600', 'no_sub_title': 'Kristi Noem', 'notional_value': 100, 'notional_value_dollars': '1.0000', 'open_interest': 2741, 'open_interest_fp': '2741.00', 'open_time': '2026-02-04T15:00:00Z', 'previous_price': 0, 'previous_price_dollars': '0.0000', 'previous_yes_ask': 0, 'previous_yes_ask_dollars': '0.0000', 'previous_yes_bid': 0, 'p

In [15]:

from typing import Any, Dict, List, Optional
def fetch_market_by_ticker(
    ticker: str,
    timeout_s: int = 10,
) -> Dict[str, Any]:
    """
    Fetch a single market by its ticker.
    Returns raw JSON (dict). The market dict is usually in response["market"].
    """
    t = (ticker or "").strip()
    if not t:
        raise ValueError("ticker is empty")

    url = f"{BASE_URL}/markets/{t}"
    resp = requests.get(url, timeout=timeout_s)
    resp.raise_for_status()
    return resp.json()

In [28]:
response = fetch_market_by_ticker("KXNBAGAME-26FEB04DENNYK-DEN")

In [32]:
def build_list_title(m: dict) -> str:
    base = (m.get("title") or "").strip()

    strike = m.get("floor_strike")
    strike_type = m.get("strike_type")

    # Numeric thresholds
    if strike is not None and strike_type in {"greater", "less", "greater_equal", "less_equal"}:
        print("HITTT\n")
        sym = {
            "greater": ">",
            "less": "<",
            "greater_equal": "≥",
            "less_equal": "≤",
        }[strike_type]
        return f"{base} — {sym} {strike}"

    # If YES label clarifies the market, append it
    yes_label = (m.get("yes_sub_title") or "").strip()
    if yes_label and yes_label.lower() not in base.lower():
        return f"{base} — {yes_label}"

    return base

In [33]:
market = response["market"]
print(build_list_title(market))

Denver at New York Winner?


In [31]:
print(market)

{'can_close_early': True, 'close_time': '2026-02-19T00:00:00Z', 'created_time': '2026-02-02T14:01:22.990309Z', 'custom_strike': {'basketball_team': 'ba2cb6be-9827-46a8-8e6f-d8661307ba77'}, 'early_close_condition': 'This market will close and expire after a winner is declared.', 'event_ticker': 'KXNBAGAME-26FEB04DENNYK', 'expected_expiration_time': '2026-02-05T03:00:00Z', 'expiration_time': '2026-02-19T00:00:00Z', 'expiration_value': '', 'last_price': 40, 'last_price_dollars': '0.4000', 'latest_expiration_time': '2026-02-19T00:00:00Z', 'liquidity': 248804569, 'liquidity_dollars': '2488045.6900', 'market_type': 'binary', 'no_ask': 64, 'no_ask_dollars': '0.6400', 'no_bid': 61, 'no_bid_dollars': '0.6100', 'no_sub_title': 'Denver', 'notional_value': 100, 'notional_value_dollars': '1.0000', 'open_interest': 3066223, 'open_interest_fp': '3066223.00', 'open_time': '2026-02-02T19:05:00Z', 'previous_price': 35, 'previous_price_dollars': '0.3500', 'previous_yes_ask': 35, 'previous_yes_ask_dollars